# Small Problems


Recursively


In [1]:
def fibonacci(n: int) -> int:
    if n < 2: return n
    return fibonacci(n -1) +fibonacci(n - 2)

In [2]:
fibonacci(10)

55

In [3]:

%%timeit 
fibonacci(10)

19.7 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:

%%prun
fibonacci(20)

# Using Memoization


In [5]:

from typing import Dict
memo: Dict[int, int] = {0: 0, 1: 1}
    
def fibo_with_memoization(n: int) -> int:
    if n not in memo:
        memo[n] = fibo_with_memoization(n - 1) + fibo_with_memoization(n - 2)
    return memo[n]

In [6]:
fibo_with_memoization(10)


55

In [7]:
%%timeit 
fibo_with_memoization(10)

133 ns ± 8.59 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [8]:
memo

{0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34, 10: 55}

In [9]:
fibo_with_memoization(11)


89

In [10]:
memo: Dict[int, int] = {0: 0, 1: 1}


In [11]:
%%prun 
fibo_with_memoization(20)

In [12]:

%%prun
fibo_with_memoization(21)

# Using lru_cache

In [13]:
from functools import lru_cache


In [14]:
@lru_cache(maxsize=None)
def fibo_with_lru_cache(n: int) -> int:
    if n < 2: return n
    return fibo_with_lru_cache(n - 1) + fibo_with_lru_cache(n - 2)

In [15]:
fibo_with_lru_cache(10)


55

In [16]:
%%timeit 
fibo_with_lru_cache(10)

72 ns ± 3.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [17]:
%%prun
fibo_with_lru_cache(10)

# Iteratively

In [18]:
def fibo_iterative(n: int) -> int:
    if n == 0: return n
    last: int = 0
    next: int = 1
    for i in range(1, n):
        last, next = next, next + last
    return next

In [19]:
fibo_iterative(10)


55

In [20]:

%%timeit 
fibo_iterative(10)

699 ns ± 32.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [21]:
%%prun
fibo_iterative(10)


# Generator Function

In [22]:
def fibo_iterative_gen(n: int) -> int:
    yield 0
    if n == 0: return 1
    last: int = 0
    next: int = 1
    for i in range(1, n):
        last, next = next, next + last
        yield next

In [23]:
list(fibo_iterative_gen(10))


[0, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [24]:
list(fibo_iterative_gen(20))


[0,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765]

# Compression

In [25]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)

    def _compress(self, gene: str) -> None:
        self.bit_string: int = 1
        for nucleotide in gene.upper():
            self.bit_string <<= 2
            if nucleotide == "A":
                self.bit_string |= 0b00
            elif nucleotide == "C":
                self.bit_string |= 0b01
            elif nucleotide == "G":
                self.bit_string |= 0b10
            elif nucleotide == "T":
                self.bit_string |= 0b11
            else:
                raise ValueError(f"Invalid nucleotide: {nucleotide}")
    
    def decompress(self) -> str:
        gene: str = ''
        for i in range(0, self.bit_string.bit_length() - 1, 2):
            bits: int = self.bit_string >> i & 0b11
            if bits == 0b00:
                gene += "A"
            elif bits == 0b01:
                gene += "C"
            elif bits == 0b10:
                gene += "G"
            elif bits == 0b11:
                gene += "T"
            else:
                raise ValueError(f"Invalid bits: {bits}")
        
        return gene[::-1]

    def __str__(self) -> str:
        return self.decompress()

In [26]:
cg = CompressedGene('ATATGCGC')


In [27]:
cg.bit_string


78745

In [28]:
str(cg)


'ATATGCGC'

In [29]:
CompressedGene('C').bit_string


5

In [30]:
import sys
import numpy as np

test_string = ''.join(list(np.random.choice(['A', 'C', 'T', 'G'], size=100, replace=True)))

print(f'Size of original string = {sys.getsizeof(test_string)}')

cg = CompressedGene(test_string)
compressed_string = cg.bit_string
print(f'Size of compressed bit string = {sys.getsizeof(compressed_string)}')

check = cg.decompress() == test_string

print(f'The original string is matched by the compressed and decompressed string: {check}.')

Size of original string = 149
Size of compressed bit string = 52
The original string is matched by the compressed and decompressed string: True.


# Unbreakable Encryption

In [31]:
test_string.encode()


b'AGTTTAAAAGAGTATATCCAGGAAAATATCAGCGGCTCTGCACCTGTGCGGCCGTGAAACCCAAGTCGTCAACTAAACCCCCACTCTTTGTTCCTGTGTT'

In [32]:

from secrets import token_bytes
from typing import Tuple

def random_key(length: int) -> int:
    tb: bytes = token_bytes(length)
    return int.from_bytes(tb, 'big')

In [33]:
random_key(2)


43784

In [34]:
def encrypt(original: str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key ^ dummy
    return dummy, encrypted

In [35]:
dummy, encrypted = encrypt(test_string)


In [36]:
int.from_bytes(test_string.encode(), 'big')


1700308009494431312541077136337359045146114519946731910395755884849435597815466493303493042627545146169078867840888647723081768884463851854816008903687159913418110015685826832437418015813451112588322390553283323229012800835348907100637451348

In [37]:
12 ^ 10


6

In [38]:
# Avoid an off by 1 error by adding 7
(dummy ^ encrypted).to_bytes((dummy.bit_length() + 7) // 8, 'big')

b'AGTTTAAAAGAGTATATCCAGGAAAATATCAGCGGCTCTGCACCTGTGCGGCCGTGAAACCCAAGTCGTCAACTAAACCCCCACTCTTTGTTCCTGTGTT'

In [39]:
len(test_string)


100

In [40]:

def decrypt(dummy: int, encrypted: int) -> str:
    decrypted_int: int = dummy ^ encrypted
    return decrypted_int.to_bytes((decrypted_int.bit_length() + 7) // 8, 'big').decode()

In [41]:
from string import ascii_uppercase

test_string = ''.join(list(np.random.choice([letter for letter in ascii_uppercase], size=10000, replace=True)))

In [42]:

print(f'Size of original string = {sys.getsizeof(test_string)}')

dummy, encrypted = encrypt(test_string)

print(f'Size of encrypted integer = {sys.getsizeof(encrypted)}')

decrypted = decrypt(dummy, encrypted)
check = test_string == decrypted
print(f'Original string was encrypted and decrypted: {check}.')

Size of original string = 10049
Size of encrypted integer = 10692
Original string was encrypted and decrypted: True.


# Calculating PI

In [43]:
def calculate_pi(n: int) -> float:
    """
    :param n: number of terms
    
    :return pi: value of pi at n terms
    """
    total = 0
    numerator = 4
    increment = 1
    for i in range(n):
        denominator = increment if i % 2 == 0 else -increment
        increment += 2
        total += numerator/denominator

    return total

In [44]:
calculate_pi(100)


3.1315929035585537

In [45]:
%%timeit 
calculate_pi(1000)

156 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [46]:
calculate_pi(1000)


3.140592653839794

In [47]:
memo = {1:4}

def calculate_pi_with_memoization(n: int) -> float:
    """
    :param n: number of terms
    
    :return pi: value of pi at n terms
    """
    
    if n in memo.keys(): return memo[n]
    
    highest_n = max(memo.keys())
    
    total = memo[highest_n]
    
    numerator = 4
    increment = 1 + 2 * highest_n
    
    for i in range(highest_n + 1, n+1):
        denominator = -increment if i % 2 == 0 else increment
        total += numerator/denominator
        increment += 2
        memo[i] = total
    return total

In [48]:
calculate_pi_with_memoization(1)


4

In [49]:
memo

{1: 4}

In [50]:
calculate_pi_with_memoization(2)


2.666666666666667

In [51]:
memo

{1: 4, 2: 2.666666666666667}

In [52]:
calculate_pi_with_memoization(100)


3.1315929035585537

In [53]:
memo = {1:4}


In [54]:
%%timeit 
calculate_pi_with_memoization(1000)

324 ns ± 43.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [55]:
memo = {1:4}


In [56]:

%%timeit 
calculate_pi_with_memoization(1000)

227 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [57]:
%%prun
calculate_pi_with_memoization(1000)

# The Towers of Hanoi



# Stack = Last In First Out

In [58]:

from typing import TypeVar, Generic, List
T = TypeVar('T')

class Stack(Generic[T]):

    def __init__(self) -> None:
        self._container: List[T] = []

    def push(self, item: T) -> None:
        self._container.append(item)

    def pop(self) -> T:
        return self._container.pop()

    def __repr__(self) -> str:
        return repr(self._container)

In [59]:
num_discs: int = 3
tower_a: Stack[int] = Stack()
tower_c: Stack[int] = Stack()
tower_b: Stack[int] = Stack()
    
for i in range(1, num_discs+1):
    tower_a.push(i)

In [60]:
tower_a


[1, 2, 3]

In [61]:
def hanoi(begin: Stack[int], end: Stack[int], temp: Stack[int], n: int) -> None:
    if n ==  1:
        end.push(begin.pop())
    else:
        hanoi(begin, temp, end, n-1)
        hanoi(begin, end, temp, 1)
        hanoi(temp, end, begin, n - 1)

In [62]:
hanoi(tower_a, tower_b, tower_c, num_discs)


In [63]:
tower_a, tower_b, tower_c


([], [1, 2, 3], [])

In [64]:

num_discs: int = 8
tower_a: Stack[int] = Stack()
tower_c: Stack[int] = Stack()
tower_b: Stack[int] = Stack()
    
for i in range(1, num_discs+1):
    tower_a.push(i)

In [65]:
tower_a, tower_b, tower_c


([1, 2, 3, 4, 5, 6, 7, 8], [], [])

In [66]:
hanoi(tower_a, tower_b, tower_c, num_discs)


In [67]:
tower_a, tower_b, tower_c



([], [1, 2, 3, 4, 5, 6, 7, 8], [])

# Recursive Exponentiation

In [68]:
def recursive_expo(x: int, n: int) -> int:
    if n == 0: return 1
    return x * recursive_expo(x, n - 1)


In [69]:
recursive_expo(2, 8)


256

In [70]:
@lru_cache(maxsize=None)
def recursive_expo_with_lru_cache(x: int, n: int) -> int:
    if n == 0: return 1
    return x * recursive_expo_with_lru_cache(x, n - 1)

In [71]:
recursive_expo_with_lru_cache(2, 8)


256

In [72]:
%%timeit 
recursive_expo(2, 12)

2.08 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [73]:
%%timeit
recursive_expo_with_lru_cache(2, 12)

135 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [74]:

def iterative_expo(x: int, n: int) -> int:
    if n == 0: return x
    total = x
    
    for i in range(1, n):
         total *= x
    return total

In [75]:
iterative_expo(2, 5)


32

In [76]:
%%timeit 
iterative_expo(2, 12)

942 ns ± 179 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [77]:
def faster_recursive_expo(x: int, n: int) -> int:
    if n == 0: return 1
    elif n % 2 == 0:
        return faster_recursive_expo(x ** 2, n / 2)
    else:
        return x * faster_recursive_expo(x ** x, (n - 1) / 2)

In [78]:
faster_recursive_expo(2, 8)


256

In [ ]:

%%timeit
faster_recursive_expo(2, 12)